# Model Fitting: ESS (Elliptical Slice Sampling)

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats
# import scipy.stats.norm as norm
import pandas as pd
import pymc as pm

import re
import numpy as np
import pickle as pickle

In [2]:
player_info = pd.read_csv("clean-data/player_info_pergame.csv")
game_outcomes = pd.read_csv("clean-data/game_outcomes_15-16.csv")

In [3]:
N_players = len(player_info)
N_teams = np.max(game_outcomes['Visitor_Index'].values) + 1
N_games =  len(game_outcomes)

print N_players, N_teams, N_games

476 30 1230


In [4]:
# # print np.max(game_outcomes['Visitor_Index'].values)
# # print np.max(game_outcomes['Home_Index'].values)

# a = np.array([[1,2,3,4], [1,2,3,4]])
# print a.shape
# print a.mean(1)
# print a[0][0:]

# b = np.zeros(a.shape)
# print b

In [5]:
################# INPUT SECTION ###################
###################################################

########### player indicator for each game ###############
### host_lineup_arr & guest_lineup_arr are (1230 x 476)###
with open("clean-data/host_team_line_up.pkl", "rb") as f:
    host_team_line_up = pickle.load(f)

with open("clean-data/guest_team_line_up.pkl", "rb") as f:
    guest_team_line_up = pickle.load(f)
    
# print len(host_team_line_up[0])
host_lineup_arr = np.array(host_team_line_up)
guest_lineup_arr = np.array(guest_team_line_up)


######### team indicator for each game ########### 
### host_matrix & guest_matrix are (1230 x 30) ###
guest_matrix = np.zeros((game_outcomes.shape[0], np.max(game_outcomes['Visitor_Index']) + 1), dtype = bool)
guest_matrix.shape
host_matrix = np.copy(guest_matrix)

def make_matrix(mat, indices):
    for (i, ind) in enumerate(indices):
        mat[i, ind] = True

make_matrix(host_matrix, game_outcomes['Visitor_Index'].values)
make_matrix(guest_matrix, game_outcomes['Home_Index'].values)


############## Observed data ##################
score_diff = game_outcomes['diff'].values
off_rating = player_info['PTS'].values + player_info['AST'].values
def_rating = player_info['BLK'].values + player_info["STL"].values + player_info['DRB'].values

In [6]:
def split_params(coefs, nplayers, nteams):
    '''
    Split the parameters
    first are the beta0 for each team
    then the beta for each player
    then the gamma0 for each team
    then the gamma for each player'''
    assert(coefs.shape == (2*(nplayers+nteams),))
    
    beta0 = coefs[:nteams]
    beta_player = coefs[nteams:(nplayers + nteams)]
    gamma0 = coefs[(nplayers + nteams):(nplayers + 2*nteams)]
    gamma_player = coefs[(nplayers + 2*nteams):]
    
    # parameterize sigma by its log
#     logsigma = coefs[-1]
    
    assert(beta0.shape == (nteams,))
    assert(beta_player.shape == (nplayers,))
    assert(beta0.shape == (nteams,))
    assert(gamma_player.shape == (nplayers,))
    
    return (beta0, beta_player, gamma0, gamma_player)

In [7]:

def diff_score_calc(coefs):
    # This function returns the differential score for each game given the states of the latent vairables
    
    beta0, betas, gamma0, gammas = \
        split_params(coefs, N_players, N_teams)
        
    guest_off_0 = np.dot(guest_matrix, beta0)
    guest_def_0 = np.dot(guest_matrix, gamma0)
    host_off_0 = np.dot(host_matrix, beta0)
    host_def_0 = np.dot(host_matrix, gamma0)
    
    guest_off = guest_off_0 + np.dot(guest_lineup_arr, betas * off_rating)
    guest_def = guest_def_0 + np.dot(guest_lineup_arr, gammas * def_rating)
    host_off = host_off_0 + np.dot(host_lineup_arr, betas * off_rating)
    host_def = host_def_0 + np.dot(host_lineup_arr, gammas * def_rating)
    
    mean = guest_off - host_def - (host_off - guest_def)
    return mean

In [8]:
def logLik(coefs, lam = 0, sigma = 10.):
    # This functino calculates the logLikelihood for the entire games
    mean = diff_score_calc(coefs)
    loglik = np.sum(scipy.stats.norm.logpdf(score_diff, mean, sigma))
    
    # add L2 regularization
    loglik -= lam * np.dot(coefs, coefs)
    return (loglik)

In [9]:
def changeCoefs(coefs, theta, norm_random):
    ''' evaluate the negative log likelihood for our basketball model '''
    
    beta0, betas, gamma0, gammas = \
            split_params(coefs, N_players, N_teams)    
    
    norm_beta0, norm_betas, norm_gamma0, norm_gammas = \
            split_params(norm_random, N_players, N_teams)
    
#     nteams = len(beta0)
#     nplayers = len(betas)
    
    new_coefs = np.zeros(coefs.shape)    
    new_coefs[:nteams] = beta0 * np.cos(theta) + norm_beta0 * np.sin(theta)
    new_coefs[nteams:(nplayers + nteams)] = betas * np.cos(theta) + norm_betas * np.sin(theta)
    new_coefs[(nplayers + nteams):(nplayers + 2*nteams)] = gamma0 * np.cos(theta) + norm_gamma0 * np.sin(theta)
    new_coefs[(nplayers + 2*nteams):] = gammas * np.cos(theta) + norm_gammas * np.sin(theta)
    
    return new_coefs

In [10]:
nplayers = player_info.shape[0]
nteams = host_matrix.shape[1]
# nteams = N_teams

# coefs = np.random.randn(2*(nplayers + nteams))
# b0, bs, g0, gs = split_params(coefs, nplayers, nteams)

# host_lineup_arr = np.array(host_team_line_up)
# guest_lineup_arr = np.array(guest_team_line_up)

In [11]:
# a = np.random.rand(5)
# temp = np.zeros((5,5))
# temp[0,:] = a
# print a
# print temp

# mvn = np.random.multivariate_normal

# print mvn(np.zeros(3), np.identity(3), 5)

# a = np.array([1,2,3,4])
# b = np.array([1,2,3,4])
# print a/b

In [12]:
mvn = np.random.multivariate_normal
def ess(logLik, N_mcmc, burn_in):
    bi = burn_in

    # Make a random initial proposal for the states of the latent variables
    # based on the assumption that the states are in the Gaussian distribution
    mcmc_coefs = np.random.randn(N_mcmc + bi, 2*(N_players + N_teams))
          
    dim = 2*(N_teams + N_players)
    norm_random = mvn(np.zeros(dim), np.identity(dim), N_mcmc+bi)
    
    # random draw from unifrom distribution with which we'll determine 
    # the loglikelihood threshold (likelihood threshold defines the 'slice' where we sample)
    unif_samples = np.random.uniform(0, 1, N_mcmc+bi)
    
    # initial proposal of the theta
    theta = np.random.uniform(0, 2*np.pi, N_mcmc+bi)
    
    # variables with which we'll propose a new state by shrinking the range of theta
    theta_min = theta - 2*np.pi
    theta_max = theta + 2*np.pi
    
    # We select a new location (i.e. new state of the latent variables)
    # on the randomly generated ellipse given theta and norm_samples
    for i in range(1, N_mcmc + bi):
#         if i % 100 == 0:
#             print i
     
        coefs = mcmc_coefs[i - 1, :]

        # the loglikelihood threshold
        # the threshold is chosen between [0, Likelihood]
        llh_thresh = logLik(coefs) + np.log(unif_samples[i])
        
        # new proposals for the latent variable states
        new_coefs = changeCoefs(coefs, theta[i], norm_random[i,:])
        while logLik(new_coefs) < llh_thresh:
            if theta[i] < 0:
                theta_min[i] = theta[i]
            else:
                theta_max[i] = theta[i]

            # shrink the range of the search for the new proposal while we logLikelohood > threshold
            theta[i] = np.random.uniform(theta_min[i], theta_max[i], 1)  
            
            # new proposals with the changed range of the ellipse
            new_coefs = changeCoefs(coefs, theta[i], norm_random[i,:])
        
        # when the condition is satisfied, we keep the proposals of the latent variable states
        mcmc_coefs[i, :] = new_coefs

    # return except the burnin     
    return mcmc_coefs[(bi + 1): (bi + N_mcmc),]

# Estimation of the latent variable states

In [14]:
# game_index = 10
N_mcmc = 2000
burn_in = 100

start_time = time.time()

stateCoefs = ess(logLik, N_mcmc, burn_in)

elapsed = time.time() - start_time
print("Elapsed Time (sec): %f" %elapsed)

mean_coefs = stateCoefs.mean(axis=0)
mean_b0, mean_bs, mean_g0, mean_gs = split_params(mean_coefs, N_players, N_teams)

print mean_b0.shape 
print mean_bs.shape
print mean_g0.shape
print mean_gs.shape

Elapsed Time (sec): 157.600000
(30L,)
(476L,)
(30L,)
(476L,)


In [15]:
contribute_off = np.argsort(-mean_bs)# * player_info['PTS'])

betas_df = player_info.loc[contribute_off]
betas_df['beta'] = mean_bs[contribute_off]
betas_df.head(20)

,Player,Tm,2015-16,Pos,Age,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,beta
170,Alexis Ajinca,New Orleans Pelicans,4300000,C,27,59,17,14.6,2.5,5.3,...,1.3,3.3,4.6,0.5,0.3,0.6,0.9,2.3,6.0,2.549573
167,Mike Dunleavy,Chicago Bulls,4500000,SF,35,31,30,22.7,2.5,6.1,...,0.3,2.4,2.7,1.3,0.5,0.3,0.8,2.1,7.2,2.505972
85,Kevin Garnett,Minnesota Timberwolves,8500000,PF,39,38,38,14.6,1.4,3.0,...,0.4,3.6,3.9,1.6,0.7,0.3,0.4,1.8,3.2,2.496795
80,Alec Burks,Utah Jazz,9213484,SG,24,31,3,25.7,4.4,10.8,...,0.5,3.0,3.5,2.0,0.6,0.1,1.6,2.3,13.3,2.495629
37,Draymond Green,Golden State Warriors,14300000,PF,25,81,81,34.7,5.0,10.1,...,1.7,7.8,9.5,7.4,1.5,1.4,3.2,3.0,14.0,2.480363
315,David West,San Antonio Spurs,1499000,PF,35,78,19,18.0,3.1,5.7,...,0.9,3.0,4.0,1.8,0.6,0.7,0.9,1.8,7.1,2.330315
68,Marcin Gortat,Washington Wizards,11217391,C,31,75,74,30.1,5.8,10.2,...,3.0,6.9,9.9,1.4,0.6,1.3,1.6,2.6,13.5,2.259320
270,Miles Plumlee,Milwaukee Bucks,2109294,C,27,61,14,14.3,2.3,3.8,...,1.5,2.3,3.8,0.3,0.3,0.8,0.7,1.2,5.1,2.108766
141,Josh Smith,Detroit Pistons,5400000,PF,30,55,7,16.0,2.3,6.3,...,0.9,2.6,3.5,1.6,0.6,0.9,1.4,2.1,6.0,2.106421
81,Jeff Green,Los Angeles Clippers,9200000,SF,29,80,41,28.2,4.4,10.3,...,0.9,3.2,4.2,1.7,0.7,0.5,1.2,2.1,11.7,2.065251


In [16]:
betas_df[betas_df['Tm'] == "New Orleans Pelicans"]

,Player,Tm,2015-16,Pos,Age,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,beta
170,Alexis Ajinca,New Orleans Pelicans,4300000,C,27,59,17,14.6,2.5,5.3,...,1.3,3.3,4.6,0.5,0.3,0.6,0.9,2.3,6.0,2.549573
473,Nate Robinson,New Orleans Pelicans,44000,PG,31,2,1,11.5,0.0,0.5,...,0.0,0.0,0.0,2.0,0.5,0.0,0.0,2.5,0.0,1.538746
67,Tyreke Evans,New Orleans Pelicans,11227000,SG,26,25,25,30.6,5.4,12.6,...,0.8,4.4,5.2,6.6,1.3,0.3,2.9,2.6,15.2,1.532877
348,Toney Douglas,New Orleans Pelicans,1164858,PG,29,61,18,20.7,3.0,7.3,...,0.4,1.9,2.3,2.6,1.1,0.1,1.0,2.1,8.7,1.072515
304,Kendrick Perkins,New Orleans Pelicans,1499200,C,31,37,5,14.6,1.1,2.0,...,0.7,2.8,3.5,0.8,0.3,0.3,1.1,1.7,2.5,1.031074
71,Jrue Holiday,New Orleans Pelicans,10595507,PG,25,65,23,28.2,6.3,14.4,...,0.4,2.6,3.0,6.0,1.4,0.3,2.6,2.3,16.8,1.024446
222,Dante Cunningham,New Orleans Pelicans,3000000,SF,28,80,46,24.6,2.3,5.2,...,0.7,2.2,3.0,1.0,0.5,0.4,0.5,2.2,6.1,0.876138
216,Norris Cole,New Orleans Pelicans,3036927,PG,27,45,23,26.6,4.4,10.8,...,0.2,3.1,3.4,3.7,0.8,0.1,1.7,2.3,10.6,0.557321
365,Luke Babbitt,New Orleans Pelicans,1100000,SF,26,47,13,18.0,2.6,6.1,...,0.5,2.6,3.1,1.1,0.2,0.1,0.5,1.9,7.0,-0.104882
105,Anthony Davis,New Orleans Pelicans,7070730,PF,22,61,61,35.5,9.2,18.6,...,2.1,8.1,10.3,1.9,1.3,2.0,2.0,2.4,24.3,-0.416855


In [17]:
# diff. score given estiamted coefficients
estimated = diff_score_calc(mean_coefs)

# difference between the estimatino and observed data
diff = score_diff - estimated

print estimated
print score_diff
print diff.shape
print diff
print 100.*diff/score_diff

# print 'the actual differential score = %s' %(real)
# print 'the differential score with the estimated variable states = %s' %(estimated)
# print 'difference is %s percent' %(np.fabs(real - estimated)*100.0/real)

[ -3.73434872   6.27702626 -14.58508205 ...,  10.84106253  -3.87634704
   4.53095297]
[ 12.  -2. -16. ...,  -9.  -8. -11.]
(1230L,)
[ 15.73434872  -8.27702626  -1.41491795 ..., -19.84106253  -4.12365296
 -15.53095297]
[ 131.11957269  413.85131308    8.84323716 ...,  220.45625037   51.54566198
  141.19048154]
